最初に'output_video.mp4' という名前の動画ファイルをフォルダに入れてください

参照：https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/vision-background-removal

## Prerequisites
[back to top ⬆️](#Table-of-contents:)

install required dependencies

In [1]:
%pip install -q torch torchvision pillow huggingface_hub "openvino>=2024.0.0" matplotlib "gradio>=4.15" "transformers>=4.39.1" tqdm --extra-index-url https://download.pytorch.org/whl/cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install opencv-python ipywidgets

Download model code from HuggingFace hub

## Load PyTorch model
[back to top ⬆️](#Table-of-contents:)

For loading model using PyTorch, we should use `AutoModelForImageSegmentation.from_pretrained` method. Model weights will be downloaded automatically during first model usage. Please, note, it may take some time.

In [1]:
from transformers import AutoModelForImageSegmentation

net = AutoModelForImageSegmentation.from_pretrained("briaai/RMBG-1.4", trust_remote_code=True)

## Convert Model to OpenVINO Intermediate Representation format

In [2]:
import torch
import torch.nn.functional as F
from torchvision.transforms.functional import normalize
import numpy as np

def preprocess_image(im: np.ndarray, model_input_size: list) -> torch.Tensor:
    if len(im.shape) < 3:
        im = im[:, :, np.newaxis]
    # orig_im_size=im.shape[0:2]
    im_tensor = torch.tensor(im, dtype=torch.float32).permute(2,0,1)
    im_tensor = F.interpolate(torch.unsqueeze(im_tensor,0), size=model_input_size, mode='bilinear').type(torch.uint8)
    image = torch.divide(im_tensor,255.0)
    image = normalize(image,[0.5,0.5,0.5],[1.0,1.0,1.0])
    return image


def postprocess_image(result: torch.Tensor, im_size: list)-> np.ndarray:
    result = torch.squeeze(F.interpolate(result, size=im_size, mode='bilinear') ,0)
    ma = torch.max(result)
    mi = torch.min(result)
    result = (result-mi)/(ma-mi)
    im_array = (result*255).permute(1,2,0).cpu().data.numpy().astype(np.uint8)
    im_array = np.squeeze(im_array)
    return im_array

In [5]:
import openvino as ov
from pathlib import Path
from PIL import Image
from transformers import AutoModelForImageSegmentation
import cv2

net = AutoModelForImageSegmentation.from_pretrained("briaai/RMBG-1.4", trust_remote_code=True)

ov_model_path = Path("rmbg-1.4.xml")
model_input_size = [1024, 1024]

cap = cv2.VideoCapture("output_video.mp4")
if not cap.isOpened():
    print("動画ファイルを開けませんでした。")
    exit()
ret, frame = cap.read()
cap.release()

image = preprocess_image(frame, model_input_size)

if not ov_model_path.exists():
    ov_model = ov.convert_model(net, example_input=image, input=[1, 3, *model_input_size])
    ov.save_model(ov_model, ov_model_path)

<class 'numpy.ndarray'>


## Run OpenVINO model inference
[back to top ⬆️](#Table-of-contents:)

After finishing conversion, we can compile converted model and run it using OpenVINO on specified device.
For selection inference device, please use dropdown list below:

In [6]:
import openvino as ov
import ipywidgets as widgets

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="NPU",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'NPU', 'AUTO'), value='NPU')

Let's run model on the same image that we used before for launching PyTorch model. OpenVINO model input and output is fully compatible with original pre- and postprocessing steps, it means that we can reuse them.

In [7]:
import cv2
from transformers import AutoModelForImageSegmentation
 
ov_compiled_model = core.compile_model(ov_model_path, device.value)


In [8]:
from PIL import Image

cap = cv2.VideoCapture(r'output_video.mp4')

if (cap.isOpened()== False):  
  print("ビデオファイルを開くとエラーが発生しました") 


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
model_input_size = [1024, 1024]

orig_im_size = [height, width]

no_bg_image = Image.new("RGBA", orig_im_size, (0, 0, 0, 0))

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret:
        image = preprocess_image(np.array(frame), model_input_size)
        
        #result = ov_compiled_model(torch.from_numpy(image))[0]
        result = ov_compiled_model(image)[0]

        # post process
        result_image = postprocess_image(torch.from_numpy(result), orig_im_size)
        pil_im = Image.fromarray(result_image)

        ori_image = Image.fromarray(frame)
        no_bg_image = Image.new("RGBA", [width, height], (0, 0, 0, 0))
        no_bg_image.paste(ori_image, mask=pil_im)

        # Play
    
        cv2.imshow("Video", np.array(no_bg_image))
        
        if cv2.waitKey(25) & 0xFF == ord('q'): 
            break
    
    else:
        break

cap.release()

cv2.destroyAllWindows()
